In [68]:
from scipy.stats import binomtest
from scipy.stats import binom
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

end_date = datetime(year=2024, month=1, day=1)
start_date = datetime(year=2023, month=1, day=1)


ticker = "INFY.NS"


df_single = yf.download(
    tickers=ticker,
    start=start_date,
    end=end_date,
    interval="1d",
    auto_adjust=False,
    progress=False
)

close_prices = df_single["Close"]
print(close_prices.head())

df_single = df_single[['Close']].dropna()

df_single['Daily_Return'] = df_single['Close'].pct_change() * 100

df_single['up_down'] = None



Ticker          INFY.NS
Date                   
2023-01-02  1524.000000
2023-01-03  1522.550049
2023-01-04  1494.800049
2023-01-05  1475.199951
2023-01-06  1448.500000


In [69]:
up_days = 0

Part 1: Theory vs Reality

In [70]:
for i in range(1, len(df_single)):
    daily_return = df_single['Daily_Return'].iloc[i]

    if pd.notna(daily_return):
        if daily_return > 0:
            df_single.at[df_single.index[i], 'up_down'] = 'up'
            up_days += 1
        else:
            df_single.at[df_single.index[i], 'up_down'] = 'down'

total_days = len(df_single) - 1
prob_up = up_days / total_days




print(f"Probability of UP days: {prob_up:.2f}")

Probability of UP days: 0.48


In [71]:
k = 117
n = 244
P = 0.5

result = binomtest(k, n, P, alternative='two-sided')
print(f"P-value: {result.pvalue:.4f}")

P-value: 0.5646


To check the probability of up day is 0.5. for  daily returns of Infosys stocks.


We counted:

Total trading days: 244

Up days: 117

So probability = 177/244 = 0.48


then binomial test
P= 0.5646 → This indicates no significant difference .

PART2

In [72]:
df_single = df_single.reset_index()
fig = make_subplots(rows=1, cols=1, subplot_titles=['Daily Returns'])

fig.add_trace(
    go.Scatter(x=df_single['Date'], y=df_single['Daily_Return'], name='Daily Return'),
    row=1, col=1
)

fig.update_layout(height=400, title_text='Daily Returns Over Time')
fig.show()

In [73]:
print(df_single['up_down'][1:11].values)

['down' 'down' 'down' 'down' 'up' 'down' 'up' 'up' 'up' 'up']


In [74]:
Return_10_day = df_single['Daily_Return'][1:11].sum()
average_10 = Return_10_day / 10

print(f"Average return over first 10 days: {average_10:.4f}")

Average return over first 10 days: 0.0200


In [75]:
return_up_total = df_single[df_single['up_down'] == 'up']['Daily_Return'].sum()
return_down_total = df_single[df_single['up_down'] == 'down']['Daily_Return'].sum()

up_count = (df_single['up_down'] == 'up').sum()
down_count = (df_single['up_down'] == 'down').sum()

avg_return_up = return_up_total / up_count if up_count > 0 else 0
avg_return_down = return_down_total / down_count if down_count > 0 else 0

print(f"\nAverage return on up days: {avg_return_up:.4f}%")
print(f"Average return on down days: {avg_return_down:.4f}%")


net_return = 6 * avg_return_up + 4 * avg_return_down



Average return on up days: 1.0843%
Average return on down days: -0.9680%


PART 3

In [76]:
n = len(df_single) - 1
observed_up_days = (df_single['up_down'] == 'up').sum()

p_empirical = observed_up_days / n

print(f"Observed Up Days: {observed_up_days}")
print(f"Empirical Probability of Up Days: {p_empirical:.4f}")

x = np.arange(0, n+1)
binomial_probs = binom.pmf(x, n, p_empirical)



Observed Up Days: 117
Empirical Probability of Up Days: 0.4795


empirical probability of up days (p) = 0.48
expected value = 2(0.48)−1=−0.04

No i would take this value becuase EV = (-)ve and for long run ,I expect to lose 4% over the long run so the bet is not favorable unless the prob of up days >= 0.6 to ensure me  a fair or positive expected outcome.
